In [ ]:
# ! git clone https://github.com/intel-isl/MiDaS.git

Cloning into 'MiDaS'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 394 (delta 58), reused 201 (delta 37), pack-reused 144
Receiving objects: 100% (394/394), 231.02 KiB | 381.00 KiB/s, done.
Resolving deltas: 100% (139/139), done.


In [ ]:
% cd /content/MiDaS/

/content/MiDaS


In [ ]:
import os
import glob
import torch
import utils
import cv2

from midas.midas_net import MidasNet
from midas.transforms import Resize, NormalizeImage, PrepareForNet

In [ ]:
model_path = None
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MidasNet(model_path, non_negative=True).to('cuda' if torch.cuda.is_available() else 'cpu')

Loading weights:  None


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


In [ ]:
backbone = model.pretrained

In [ ]:
# ! git clone https://github.com/ajithvallabai/planercnn_customimages.git

In [ ]:
% cd planercnn_customimages/

/content/MiDaS/planercnn_customimages


In [ ]:
# ! pip3 install cffi==1.11.5

In [ ]:
from models.model import *
from utils import *

In [ ]:
% cd /content/MiDaS/planercnn_customimages

/content/MiDaS/planercnn_customimages


In [ ]:
class FPN(nn.Module):
    def __init__(self, C2, C3, C4, C5, out_channels, bilinear_upsampling=False):
        super(FPN, self).__init__()
        self.out_channels = out_channels
        self.bilinear_upsampling = bilinear_upsampling
        self.C2 = C2
        self.C3 = C3
        self.C4 = C4
        self.C5 = C5
        self.P6 = nn.MaxPool2d(kernel_size=1, stride=2)
        self.P5_conv1 = nn.Conv2d(2048, self.out_channels, kernel_size=1, stride=1)
        self.P5_conv2 = nn.Sequential(
            SamePad2d(kernel_size=3, stride=1),
            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1),
        )
        self.P4_conv1 =  nn.Conv2d(1024, self.out_channels, kernel_size=1, stride=1)
        self.P4_conv2 = nn.Sequential(
            SamePad2d(kernel_size=3, stride=1),
            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1),
        )
        self.P3_conv1 = nn.Conv2d(512, self.out_channels, kernel_size=1, stride=1)
        self.P3_conv2 = nn.Sequential(
            SamePad2d(kernel_size=3, stride=1),
            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1),
        )
        self.P2_conv1 = nn.Conv2d(256, self.out_channels, kernel_size=1, stride=1)
        self.P2_conv2 = nn.Sequential(
            SamePad2d(kernel_size=3, stride=1),
            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1),
        )

    def forward(self, x):
        x = self.C2(x)
        c2_out = x
        x = self.C3(x)
        c3_out = x
        x = self.C4(x)
        c4_out = x
        x = self.C5(x)
        p5_out = self.P5_conv1(x)
        
        if self.bilinear_upsampling:
            p4_out = self.P4_conv1(c4_out) + F.upsample(p5_out, scale_factor=2, mode='bilinear')
            p3_out = self.P3_conv1(c3_out) + F.upsample(p4_out, scale_factor=2, mode='bilinear')
            p2_out = self.P2_conv1(c2_out) + F.upsample(p3_out, scale_factor=2, mode='bilinear')
        else:
            p4_out = self.P4_conv1(c4_out) + F.upsample(p5_out, scale_factor=2)
            p3_out = self.P3_conv1(c3_out) + F.upsample(p4_out, scale_factor=2)
            p2_out = self.P2_conv1(c2_out) + F.upsample(p3_out, scale_factor=2)
            pass

        p5_out = self.P5_conv2(p5_out)
        p4_out = self.P4_conv2(p4_out)
        p3_out = self.P3_conv2(p3_out)
        p2_out = self.P2_conv2(p2_out)

        ## P6 is used for the 5th anchor scale in RPN. Generated by
        ## subsampling from P5 with stride of 2.
        p6_out = self.P6(p5_out)

        return [p2_out, p3_out, p4_out, p5_out, p6_out]



In [ ]:
l1, l2, l3, l4 = backbone.layer1, backbone.layer2, backbone.layer3, backbone.layer4

In [ ]:
fpn = FPN(l1, l2, l3, l4, out_channels=256, bilinear_upsampling=True)


In [ ]:
from torchsummary import summary
print(summary(fpn, (3, 384, 384)))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2952: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5          [-1, 256, 64, 64]          16,384
       BatchNorm2d-6          [-1, 256, 64, 64]             512
              ReLU-7          [-1, 256, 64, 64]               0
            Conv2d-8          [-1, 256, 64, 64]          18,432
       BatchNorm2d-9          [-1, 256, 64, 64]             512
             ReLU-10          [-1, 256, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          65,536
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
# from config import PlaneConfig

# model = MaskRCNN(config)

RuntimeError: ignored